In [48]:
import json
import numpy as np
import os
from pprint import pprint

# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553733240'  # TFIDF (1)
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553790696'  # TFIDF
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553901953'  # FastText
# prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553982706'  # Q-only
prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554006689'  # TFIDF(Q+O)

# Set useful variables
num_options = 4
options = ['A', 'B', 'C', 'D'][:num_options]
debate_mode_to_option = {'Ⅰ': 'A', 'Ⅱ': 'B', 'Ⅲ': 'C', 'Ⅳ': 'D', None: None}

# Read HIT data
print('# HIT Files:', len(os.listdir(task_dir)))
hit_results = []
num_passed_test = 0
for hit_dir in os.listdir(task_dir):
    if hit_dir.startswith('o_'):
        continue
    num_passed_test += 1
    with open(os.path.join(task_dir, hit_dir, 'custom/data.json'), 'r') as file:
        hit_results.append(json.load(file))
        file.close()
if len(os.listdir(task_dir)) != num_passed_test:
    num_total_tested = len(os.listdir(task_dir)) - num_passed_test
    print('# Passed Test:', num_passed_test, '/', num_total_tested, '=', round((100. * num_passed_test) / num_total_tested, 2), '%')

# HIT Files: 323
# Passed Test: 114 / 209 = 54.55 %


In [49]:
num_valid_hits, num_rejected_hits, num_incomplete_hits = 0, 0, 0
metrics = {}
task_ratings = {i: 0 for i in range(11)}
durations = []
durations_by_worker = []
accuracy_by_worker = {}
max_response_freqs = []

for hit_result in hit_results:
    if ((len(hit_result['reject_reasons']) > 0) or
        (len(hit_result['block_reasons']) > 0)):
        num_rejected_hits += 1
        print('| reject_reasons:', hit_result['reject_reasons'], '| block_reasons:', hit_result['block_reasons'])
        continue
    elif hit_result['feedback'] is None:
        num_incomplete_hits += 1
        continue
    
    num_valid_hits += 1
    if (hit_result['task_rating'] is not None) and (hit_result['task_rating'].isdigit()):
        task_ratings[int(hit_result['task_rating'])] += 1
    for qtype, qtype_accuracy in hit_result['accuracy'].items():
        accuracy_by_worker[qtype] = accuracy_by_worker.get(qtype, []) + [qtype_accuracy]
    
    hit_durations = []
    response_option_counts = {option: 0 for option in options}
    responses = []
    for prompt in hit_result['data']:
        qid = prompt['sample']['qid']
        model_stance = debate_mode_to_option[prompt['sample']['debate_mode']]
        answer = prompt['sample']['eval_labels'][0]
        human_correct = (prompt['response'] == answer)
        assert answer in options, 'Answer must be in options.'
        
        # Calculate metrics
        if qid not in metrics:
            metrics[qid] = {
                option: {
                    'num': 0,
                    'num_correct': 0,
                    'num_correct_debate_mode': 0,
                    'num_incorrect_debate_mode': 0,
                    'num_correct_with_correct_debate_mode': 0,
                    'num_correct_with_incorrect_debate_mode': 0,
                    'num_debate_mode_responses': 0,
                }
                for option in ([None] if model_stance is None else options)
            }
            metrics[qid]['answer'] = answer
        prompt_metrics = metrics[qid][model_stance]
        prompt_metrics['num'] += 1
        prompt_metrics['num_correct'] += human_correct
        if model_stance == answer:
            prompt_metrics['num_correct_with_correct_debate_mode'] += human_correct
            prompt_metrics['num_correct_debate_mode'] += 1
        else:
            prompt_metrics['num_correct_with_incorrect_debate_mode'] += human_correct
            prompt_metrics['num_incorrect_debate_mode'] += 1
        prompt_metrics['num_debate_mode_responses'] += (prompt['response'] == model_stance)
        
        hit_durations.append(prompt['duration'] / 1000.)
        response_option_counts[prompt['response']] += 1
        responses.append(prompt['response'])
    duration = np.median(np.array(hit_durations))
    durations_by_worker.append(duration)
    durations += hit_durations
    response_options_array = np.array(list(response_option_counts.values()))
    response_options_array = response_options_array / response_options_array.sum()
    max_response_freq = response_options_array.max()
    max_response_freqs.append(max_response_freq)
    print('| Time:', round(duration, 1),
          '| Acc:', round(100 * hit_result['accuracy'][prompt_type]),
          '| Max Freq:', round(100 * max_response_freq, 1),
          '| Rate:', hit_result['task_rating'],
          '| Feedback:', hit_result['feedback'])

durations = np.array(durations)
durations_by_worker = np.array(durations_by_worker)
max_response_freqs = np.array(max_response_freqs)
durations.sort()
durations_by_worker.sort()
max_response_freqs.sort()
for qtype in accuracy_by_worker:
    accuracy_by_worker[qtype] = np.array(accuracy_by_worker[qtype])
    accuracy_by_worker[qtype].sort()
print('REJECTED:', num_rejected_hits)
print('INCOMPLETE:', num_incomplete_hits)
print('VALID:', num_valid_hits)
print('Median Question Duration:', np.median(durations))
print('Median Worker Duration:', np.median(durations_by_worker))
print('Median Worker Accuracy:', np.median(accuracy_by_worker[prompt_type]))
print('Median Max Response Freq:', np.median(max_response_freqs))
# pprint(hit_results[0]['data'][0])
# pprint(hit_results[0])

| Time: 17.2 | Acc: 68 | Max Freq: 36.8 | Rate: 7 | Feedback: can't think of anything
| Time: 28.7 | Acc: 58 | Max Freq: 31.6 | Rate: 5 | Feedback: with a bit more explanation of the intent
| Time: 15.5 | Acc: 40 | Max Freq: 30.0 | Rate: 1 | Feedback: There was an issue the screen was jumpy like there is some kind of glitch
| Time: 48.0 | Acc: 63 | Max Freq: 36.8 | Rate: 8 | Feedback: It was fun, but I wish I got more right. Don't need to change a thing. :)
| Time: 11.5 | Acc: 53 | Max Freq: 36.8 | Rate: 5 | Feedback: Be more specific with the sentences.
| Time: 24.0 | Acc: 58 | Max Freq: 36.8 | Rate: 2 | Feedback: more sensible questions 
| Time: 27.1 | Acc: 47 | Max Freq: 31.6 | Rate: 7 | Feedback: bubbles instead of pull downs
| Time: 32.7 | Acc: 42 | Max Freq: 36.8 | Rate: 10 | Feedback: Explain the correct answers
| Time: 12.9 | Acc: 63 | Max Freq: 31.6 | Rate: 9 | Feedback: Give more information in quotes
| Time: 12.3 | Acc: 74 | Max Freq: 31.6 | Rate: 10 | Feedback: nothing
| Ti

| Time: 12.1 | Acc: 65 | Max Freq: 30.0 | Rate: 8 | Feedback: Nothing really, the questions seem smooth. Some definitely feel impossible with the context given though.
| Time: 20.9 | Acc: 60 | Max Freq: 50.0 | Rate: 6 | Feedback: I thought everything ran very smoothly 
| Time: 16.7 | Acc: 70 | Max Freq: 30.0 | Rate: 9 | Feedback: I didn't encounter any issues with the task.
| Time: 18.3 | Acc: 65 | Max Freq: 35.0 | Rate: 7 | Feedback: Nothing I can think of
| Time: 15.1 | Acc: 37 | Max Freq: 42.1 | Rate: 3 | Feedback: Not sure.
| Time: 20.8 | Acc: 74 | Max Freq: 31.6 | Rate: 10 | Feedback: I would like to know which questions I got wrong.
| Time: 29.3 | Acc: 47 | Max Freq: 42.1 | Rate: 10 | Feedback: I loved this task! Fun! Nothing really to improve, except maybe just add a little more time. :)
| Time: 35.5 | Acc: 58 | Max Freq: 36.8 | Rate: 10 | Feedback: Don't change much its fun! 
| Time: 24.1 | Acc: 58 | Max Freq: 31.6 | Rate: 10 | Feedback: nothing itd already awesome
| Time: 29.7

In [50]:
accuracy_by_sample = []
accuracy_by_sample_correct_debate_mode = []
accuracy_by_sample_incorrect_debate_mode = []
convinced_freqs = []
num_evals_by_sample = []
for qid, qid_metrics in metrics.items():
    answer = metrics[qid]['answer']
    for model_stance, prompt in qid_metrics.items():
        if not (model_stance in [None] + options):
            continue

        # Q-only stats
        prompt_metrics = metrics[qid][model_stance]
        num_evals_by_sample.append(prompt_metrics['num'])
        accuracy_by_sample.append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        if model_stance is None:
            continue
        
        # Debater stats
        if model_stance == answer:
            accuracy_by_sample_correct_debate_mode.append(
                prompt_metrics['num_correct_with_correct_debate_mode'] /
                prompt_metrics['num_correct_debate_mode'])
        else:
            accuracy_by_sample_incorrect_debate_mode.append(
                prompt_metrics['num_correct_with_incorrect_debate_mode'] /
                prompt_metrics['num_incorrect_debate_mode'])
        convinced_freqs.append(prompt_metrics['num_debate_mode_responses'] / prompt_metrics['num'])

num_evals_by_sample = np.array(num_evals_by_sample)
print('Evals per sample:', num_evals_by_sample.mean())
print('Fraction insuffient evals:', (num_evals_by_sample < 5).mean())
accuracy_by_sample = np.array(accuracy_by_sample)
print('Accuracy:', round(100 * accuracy_by_sample.mean(), 2), '%')

accuracy_by_sample_correct_debate_mode = np.array(accuracy_by_sample_correct_debate_mode)
print('Accuracy correct debate mode:', round(100 * accuracy_by_sample_correct_debate_mode.mean(), 2), '%')
accuracy_by_sample_incorrect_debate_mode = np.array(accuracy_by_sample_incorrect_debate_mode)
print('Accuracy incorrect debate mode:', round(100 * accuracy_by_sample_incorrect_debate_mode.mean(), 2), '%')
convinced_freqs = np.array(convinced_freqs)
print('Convinced:', round(100 * convinced_freqs.mean(), 2), '%')

num_target_evals = 5
print('Extra Evals:', round(((100. * (num_evals_by_sample - num_target_evals).sum()) / num_evals_by_sample.sum()), 2), '%')
num_evals_by_sample.sort()
print('Evals per sample distribution:', num_evals_by_sample)

Evals per sample: 5.494791666666667
Fraction insuffient evals: 0.0
Accuracy: 57.86 %
Accuracy correct debate mode: 69.26 %
Accuracy incorrect debate mode: 54.07 %
Convinced: 33.43 %
Extra Evals: 9.0 %
Evals per sample distribution: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9
 9 9 9 9 9 9 9 9 9

In [51]:
def nps(task_ratings):
    num_ratings = sum(list(task_ratings.values()))
    if num_ratings == 0:
        return None

    sum_ratings = 0
    for score, num_raters in task_ratings.items():
        if score >= 9:
            sum_ratings += num_raters
        elif score <= 6:
            sum_ratings -= num_raters
    return round(100 * (sum_ratings / float(num_ratings)), 2)

print('NPS:', nps(task_ratings))

NPS: 29.09


In [52]:
max_response_freqs

array([0.25      , 0.26315789, 0.26315789, 0.3       , 0.3       ,
       0.3       , 0.3       , 0.3       , 0.3       , 0.3       ,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.31578947, 0.31578947, 0.31578947,
       0.31578947, 0.31578947, 0.35      , 0.35      , 0.35      ,
       0.35      , 0.35      , 0.36842105, 0.36842105, 0.36842105,
       0.36842105, 0.36842105, 0.36842105, 0.36842105, 0.36842105,
       0.36842105, 0.36842105, 0.36842105, 0.36842105, 0.36842105,
       0.36842105, 0.36842105, 0.36842105, 0.36842105, 0.36842105,
       0.36842105, 0.36842105, 0.36842105, 0.36842105, 0.36842

In [55]:
accuracy_by_worker[prompt_type]

0.5789473684210527

In [54]:
np.median(durations_by_worker)

20.956249999999997